# Les arbres de régression et de classification

## I. Introduction

Les arbres de décisions forment une catégorie de modèles clé en *marchine learning*. Bien que n'étant pas les modèles les plus performants par eux-mêmes, ils composent les briques de bases des forêts aléatoires, souvent sur le podium des modèles les plus performants. De plus, leur facilité d'interprétation leur offre un atout non négligeable.

Les arbres de décisions s'appuient sur une structure de données qu'on appelle un [arbre](https://fr.wikipedia.org/wiki/Arbre_(théorie_des_graphes)). Il 'agit en particulier de ce qu'on appelle un [arbre de décision](https://fr.wikipedia.org/wiki/Arbre_de_décision). Le principe de ce dernier est illustré par la figure suivante :


![Decision tree](https://raw.githubusercontent.com/maximiliense/lmiprp/main/Travaux%20Pratiques/Machine%20Learning/Introduction/data/Introduction/cart.jpg)

Imaginons l'étudiant John Smith caractérisé par une note en licence en informatique de 14, une note en mathématiques de 15 et qui écoute en cours. Sa classification se fera comme suit.
1.  On regarde la racine : l'étudiant écoute en cours, on prend donc la branche de droite,
2.  On regarde sa note en math : elle est supérieure à 10, on en conclut qu'il obtiendra son module de *machine learning*.

L'idée est qu'à chaque nœud notre arbre va découper le sous-espace dont "il s'occupe par un hyperplan de manière à définir deux hyperrectangles. L'un sera traité par les nœuds de la branche droite et l'autre par ceux de la branche gauche.

In [ ]:
import numpy as np

def create_student_dataset(n):
    # la premiere colonne est la note d'informatique,
    # la seconde la note de math
    X = np.random.uniform(0, 20, size=(n, 2))
    conditions = np.stack([
        X[:, 0] > 11,
        X[:, 1] > 8,
        X[:, 1] > 18
    ]).T
    y = np.any(np.stack(
         [np.all(conditions[:, 0:2], axis=1), X[:, 1] > 18]), axis=0
             )
    return X, y
    
X, y = create_student_dataset(45)

In [ ]:
import matplotlib.pyplot as plt

def plot_student(X, y, tree=None):
    plt.figure(figsize=(12, 8))
    plt.scatter(X[y, 0], X[y, 1], color='green', label='Réussi le module de machine learning')
    plt.scatter(X[(1-y).astype(bool), 0], X[(1-y).astype(bool), 1], 
                c='red', label='Échoue le module de machine learning')
    plt.xlabel('Informatique')
    plt.ylabel('Mathématiques')
    plt.legend()
    plt.show()

plot_student(X, y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(
    np.arange(x_min, x_max, 0.1),
    np.arange(y_min, y_max, 0.1)
)

plt.figure(figsize=(15, 5))
for i in range(1, 4):
    ax = plt.subplot(1, 3, i)
    tree = DecisionTreeClassifier(max_depth=i)
    tree.fit(X, y)
    Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k')
    plt.xlabel('Informatique')
    plt.ylabel('Mathématiques')
    plt.title('Arbre de décision avec une profondeur de ' + str(i))
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))
_ = plot_tree(tree, rounded=True, fontsize=14)

## II. Construction d'un arbre de classification

La question importante est de définir la manière de séparer les données à chaque nœud. À la première itération, l'idée va être de considérer toutes les valeurs possibles de seuil $t$ et des variables $j$ qui séparerait notre jeu de données $S$ en deux groupes $S^{(l)}$ et $S^{(r)}$ tels que $\forall x\in S_n^{(l)}\Rightarrow x_j \leq t$ et $\forall x\in S_n^{(r)}\Rightarrow x_j > t$. Le choix du seuil se fera sur un critère qu'on appelle "le gain d'information" :

$$\mathcal{IG}(S^{l}, S^{r})=\mathcal{I}(S)-\frac{n_l}{n}\mathcal{I}(S^{l})-\frac{n_r}{n}\mathcal{I}(S^{r}),$$

où $n_r=|S^{(r)}|$ et $n_l=|S^{(l)}|$ et $\mathcal{I}$ est une "mesure d'impureté" d'un groupe que nous définierons plus tard. L'idée est de mesure à quel point notre seuil $t$ a bien séparé notre jeu de données en deux groupes où les classes sont moins mélangées que dans le jeu complet $S$.

Plusieurs métriques d'impureté existent dont nous listons quelques exemples :

* Impureté de Gini
* Entropie
* Erreur de classification

L'impureté de Gini se calcule de la manière suivante :

$$\mathcal{I}_{\mathcal{G}}(S)=\sum_i p_i(1-p_i),$$

où $p_i$ indique la proportion d'éléments de la classe $i$ dans l'ensemble $S$. L'entropie est donnée par la formule $\mathcal{I}_\mathcal{E}(S)=-\sum_i p_i \text{log}_2 p_i$ et l'erreur de classification par $\mathcal{I}_{EC}(S)=1-\text{max}_i(p_i)$. On remarque que dans les figures de l'exemple ci-dessus, nous avons utilisé l'impureté de Gini ! 

L'étape précédente est ensuite répétée pour chaque sous-groupe. Plusieurs stratégies d'arrêts sont possibles. Tout d'abord, lorsque chaque feuille ne contient qu'un seul élément et ne peut plus être divisée en deux. Où lorsque le split améliore l'*accuracy* d'un gain relatif d'au moins $\alpha$ (hyperparamètre qu'on fixe). Une autre stratégie consiste à construire l'arbre complet et à ensuite élimner les branches qui ne satisfont pas certains critères par exemple au travers d'une validation croisée.

## III. Construction d'un arbre de régression
Dans le cas d'un arbre de régression, le critère de *split* est différent du cas précédent. Nous considérons cette fois-ci comme critère d'erreur (l'impureté précédente) le *mean-squared error* ou MSE :

$$MSE = \sum_i(\bar{y} – y_i)^2/n.$$

Et le split minimisant la quantité suivante est choisie : 
$$\frac{n_l}{n}MSE(S^{(l)})+\frac{n_r}{n}MSE(S^{(r)}).$$
Ce n'est rien d'autre que la moyenne pondérée des erreurs.

## IV. Applications
---

<span style="color:blue">**Exercice :**</span> **Pour les différents jeux de données suivants, proposez un type d'arbre de classification (régression ou classification). Quel critère est split a été utilisé ? Quel est la profondeur de l'arbre calculé. Quelle règle de décision (i.e. le chemin de décision dans l'arbre) a été utilisée pour le premier élément du jeu de test (uniquement les jeux de données 1 avec max_depth=3 et 2) ?**


---

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

**Jeu de données 1**

In [ ]:
data = datasets.fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.5, shuffle=True
)

In [ ]:
####### Complete this part ######## or die ####################
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=3)
model.fit(X_train, y_train)

print('Le score R2:', model.score(X_test, y_test))

plt.figure(figsize=(16, 8))
_ = plot_tree(model, rounded=True, fontsize=10)
plt.show()

print('On prédit:', model.predict(X_test[:1]), 'pour', X_test[:1])
###############################################################

**Jeu de données 2**

In [ ]:
data = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.5, shuffle=True
)

In [ ]:
####### Complete this part ######## or die ####################
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

print('L\'accuracy de notre modèle:', model.score(X_test, y_test))

plt.figure(figsize=(12, 8))
_ = plot_tree(model, rounded=True, fontsize=14)
plt.show()
print('On prédit:', model.predict(X_test[:1]), 'pour', X_test[:1])
###############################################################

**Jeu de données 3**

In [ ]:
data = datasets.fetch_covtype()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.5, shuffle=True
)

In [ ]:
####### Complete this part ######## or die ####################
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

print('L\'accuracy de notre modèle:', model.score(X_test, y_test))

print('On prédit:', model.predict(X_test[:1]), 'pour', X_test[:1])
###############################################################